In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import random
from datetime import datetime, time

# ESTILO DE BOTONES
button_style = """
<style>
.custom-button {
    font-weight: bold;
    color: white !important;
    background-color: #88D66C !important;
    text-align: center;
    font-size: 14px;  /* Tamaño de fuente */
    padding: 5px;     /* Espaciado interno */
    margin: 5px;      /* Espaciado externo */
}
.center {
    display: flex;
    justify-content: center;
    align-items: center;
    flex-direction: column;
}
</style>
"""

# CARGO DE DATOS
url = 'https://www.datos.gov.co/resource/n4dj-8r7k.csv'
df = pd.read_csv(url)

# COLUMMNAS DE UBICACIÓN (SECCIÓN, BLOQUE,FILA)
random.seed(42)
df['sección'] = [random.randint(1, 20) for _ in range(len(df))]
df['bloque'] = [chr(random.randint(65, 74)) for _ in range(len(df))]  # Letras de A a J
df['fila'] = [random.randint(1, 20) for _ in range(len(df))]

#FUNCIÓN PARA BUSQUEDA DE MEDICAMENTOS
def buscar_medicamento(medicamento):
    resultados = df[df['medicamento'].str.contains(medicamento, case=False, na=False)]
    if resultados.empty:
        return HTML(f"<b>No se encontraron resultados para: {medicamento}</b>")
    else:
        return resultados[['principio_activo', 'unidad_base', 'unidad_de_dispensacion', 'precio_por_tableta', 'factoresprecio', 'sección', 'bloque', 'fila']]

# WIDGETS PARA LA BÚSQUEDA DE MEDICAMENTOS
medicamento_input = widgets.Text(description='Medicamento:')
buscar_button = widgets.Button(description="Buscar", layout=widgets.Layout(width='150px', height='40px', margin='10px'))
buscar_button.add_class('custom-button')
output_busqueda = widgets.Output()

# Función para mostrar la búsqueda
def buscar_medicamento_event(b):
    with output_busqueda:
        output_busqueda.clear_output()
        medicamento = medicamento_input.value
        display(buscar_medicamento(medicamento))

buscar_button.on_click(buscar_medicamento_event)

# Crear el contenedor de búsqueda
search_container = widgets.VBox([medicamento_input, buscar_button, output_busqueda], layout=widgets.Layout(align_items='center'))
search_container.layout.display = 'none'  # Ocultar el contenedor inicialmente

# Crear el botón "Catálogo"
button_catalogo = widgets.Button(description="Catálogo", layout=widgets.Layout(width='150px', height='40px', margin='10px'))
button_catalogo.add_class('custom-button')

# Función para mostrar el contenedor de búsqueda
def mostrar_catalogo(b):
    search_container.layout.display = 'block'
    atencion_container.layout.display = 'none'
    formulario_citas.layout.display = 'none'

button_catalogo.on_click(mostrar_catalogo)

# Crear contenedor para Atención al Cliente
atencion_container = widgets.VBox(layout=widgets.Layout(align_items='center'))
atencion_container.layout.display = 'none'  # Ocultar el contenedor inicialmente

# Crear botones para las opciones de Atención al Cliente
button_horario = widgets.Button(description="Horario de Atención", layout=widgets.Layout(width='200px', height='40px', margin='10px'))
button_horario.add_class('custom-button')
button_canales = widgets.Button(description="Canales de Comunicación", layout=widgets.Layout(width='200px', height='40px', margin='10px'))
button_canales.add_class('custom-button')
button_ubicacion = widgets.Button(description="Ubicación", layout=widgets.Layout(width='200px', height='40px', margin='10px'))
button_ubicacion.add_class('custom-button')

# Crear áreas de salida para mostrar la información de Atención al Cliente
output_atencion = widgets.Output()

def mostrar_horario(b):
    with output_atencion:
        output_atencion.clear_output()
        display(HTML("""
        <b>Horario de Atención:</b><br>
        - <b>Lunes a Viernes:</b> 8:00 AM - 6:00 PM<br>
        - <b>Sábado:</b> 9:00 AM - 1:00 PM<br>
        - <b>Domingo:</b> Cerrado
        """))

def mostrar_canales(b):
    with output_atencion:
        output_atencion.clear_output()
        display(HTML("""
        <b>Datos de Contacto:</b><br>
        - <b>Teléfono:</b> +57 5 345 6789<br>
        - <b>Correo Electrónico:</b> info@saludvital.com.co
        """))

def mostrar_ubicacion(b):
    with output_atencion:
        output_atencion.clear_output()
        display(HTML("""
        <b>Ubicación:</b><br>
        Carrera 50 # 80-25, Barranquilla, Atlántico, Colombia
        """))

button_horario.on_click(mostrar_horario)
button_canales.on_click(mostrar_canales)
button_ubicacion.on_click(mostrar_ubicacion)

atencion_container.children = [button_horario, button_canales, button_ubicacion, output_atencion]

# Crear el botón "Atención al Cliente"
button_atencion = widgets.Button(description="Atención al Cliente", layout=widgets.Layout(width='150px', height='40px', margin='10px'))
button_atencion.add_class('custom-button')

# Función para mostrar el contenedor de Atención al Cliente
def mostrar_atencion(b):
    search_container.layout.display = 'none'
    atencion_container.layout.display = 'block'
    formulario_citas.layout.display = 'none'

button_atencion.on_click(mostrar_atencion)

# Crear la base de datos de citas
citas_db = pd.DataFrame(columns=['nombre', 'cédula', 'teléfono', 'correo', 'fecha', 'hora'])

# Función para agregar citas a la base de datos
def agregar_cita(nombre, cédula, teléfono, correo, fecha, hora):
    global citas_db
    nueva_cita = pd.DataFrame({'nombre': [nombre], 'cédula': [cédula], 'teléfono': [teléfono], 'correo': [correo], 'fecha': [fecha], 'hora': [hora]})
    citas_db = pd.concat([citas_db, nueva_cita], ignore_index=True)

# Función para validar la fecha
def validar_fecha(fecha):
    if fecha is None:
        return False
    # Validar que la fecha sea un día de semana
    if fecha.weekday() >= 5:  # 5 y 6 son sábado y domingo
        return False
    return True

# Función para validar la hora
def validar_hora(hora):
    if hora is None:
        return False
    # Horario laboral permitido: 8:00 AM - 12:30 PM y 2:00 PM - 6:00 PM
    if not (time(8, 0) <= hora <= time(12, 30)) and not (time(14, 0) <= hora <= time(18, 0)):
        return False
    # Validar que la hora no esté repetida en la misma fecha
    if (fecha := fecha_input.value) is not None:
        if (hora_str := hora.strftime('%H:%M')) in citas_db.loc[citas_db['fecha'] == fecha, 'hora'].astype(str).values:
            return False
    return True

# Función para enviar el formulario
def enviar_cita(b):
    nombre = nombre_input.value
    cedula = cedula_input.value
    telefono = telefono_input.value
    correo = correo_input.value
    fecha = fecha_input.value
    hora = hora_input.value

    if not validar_fecha(fecha):
        with output_citas:
            output_citas.clear_output()
            display(HTML("<b>Fecha inválida. Seleccione un día de semana.</b>"))
        return

    if not validar_hora(hora):
        with output_citas:
            output_citas.clear_output()
            display(HTML("<b>Hora inválida. Seleccione una hora dentro del horario laboral permitido.</b>"))
        return

    # Verificar si ya hay una cita a esa hora y fecha
    if citas_db[(citas_db['fecha'] == fecha) & (citas_db['hora'] == hora)].shape[0] > 0:
        with output_citas:
            output_citas.clear_output()
            display(HTML("<b>Ya hay una cita agendada para esta fecha y hora. Seleccione otra.</b>"))
        return

    # Agregar cita si todo es válido
    agregar_cita(nombre, cedula, telefono, correo, fecha, hora)
    with output_citas:
        output_citas.clear_output()
        display(HTML(f"<b>Cita agendada exitosamente para el {fecha.strftime('%d/%m/%Y')} a las {hora.strftime('%H:%M')}.</b>"))

# Crear widgets para el formulario de citas
nombre_input = widgets.Text(description='Nombre:')
cedula_input = widgets.Text(description='Cédula:')
telefono_input = widgets.Text(description='Teléfono:')
correo_input = widgets.Text(description='Correo:')
fecha_input = widgets.DatePicker(description='Fecha:', disabled=False)
hora_input = widgets.Dropdown(
    options=[(f"{hour:02}:{minute:02}", datetime.combine(datetime.today(), time(hour, minute)).time()) for hour in range(8, 19) for minute in [0, 30] if not (hour == 12 and minute >= 30) and not (hour == 13)],
    description='Hora:',
    disabled=False
)

# Crear un área de salida para mostrar mensajes del formulario
output_citas = widgets.Output()

# Crear el botón de enviar
boton_enviar_cita = widgets.Button(description="Enviar Cita", layout=widgets.Layout(width='150px', height='40px', margin='10px'))
boton_enviar_cita.add_class('custom-button')

# Asignar la función al botón
boton_enviar_cita.on_click(enviar_cita)

# Crear el formulario de citas
formulario_citas = widgets.VBox([nombre_input, cedula_input, telefono_input, correo_input, fecha_input, hora_input, boton_enviar_cita, output_citas], layout=widgets.Layout(align_items='center'))
formulario_citas.layout.display = 'none'  # Ocultar el formulario inicialmente

# Crear el botón "Citas"
button_citas = widgets.Button(description="Citas", layout=widgets.Layout(width='150px', height='40px', margin='10px'))
button_citas.add_class('custom-button')

# Función para mostrar el formulario de citas
def mostrar_citas(b):
    search_container.layout.display = 'none'
    atencion_container.layout.display = 'none'
    formulario_citas.layout.display = 'block'

# Asignar la función al botón "Citas"
button_citas.on_click(mostrar_citas)

# Mostrar los botones principales
display(HTML(button_style))
display(widgets.VBox([button_catalogo, button_atencion, button_citas], layout=widgets.Layout(align_items='center')))
display(search_container)
display(atencion_container)
display(formulario_citas)
